# Importing libraries

In [ ]:
import os, glob, json, csv, subprocess, sys, re, operator
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository and directories

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/openjpa/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_openjpa/'

# Defining function to execute git command

In [ ]:
def execute_command(cmd, work_dir):
   #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Defining algorithms

In [ ]:
algorithms = ['histogram','minimal','myers','patience']

# Loading the files data for each algorithm

In [ ]:
fields = ['bug_id','bugintro_commitID','parent_id','filename','#deletions']

In [ ]:
dthist = pd.read_csv(analyze_dir + '03_file-diff/01_filename/histogram_files.csv')
dtmin = pd.read_csv(analyze_dir + '03_file-diff/01_filename/minimal_files.csv')
dtmyers = pd.read_csv(analyze_dir + '03_file-diff/01_filename/myers_files.csv')
dtpat = pd.read_csv(analyze_dir + '03_file-diff/01_filename/patience_files.csv')

dthist = dthist[fields]
dtmin = dtmin[fields]
dtmyers = dtmyers[fields]
dtpat = dtpat[fields]

In [ ]:
dthist

Make a list of dataframe

In [ ]:
data_algorithm = [dthist, dtmin, dtmyers, dtpat]

# Creating txt files of filenames list for each algorithm

In [ ]:
for da, dt_alg in enumerate(data_algorithm):
    filename = []
    for jj in range(0,len(dt_alg)):
        if dt_alg.iloc[jj]['#deletions'] != 0:
            filename.append("_" + ((dt_alg.iloc[jj][3].split('/'))[-1:])[0] + "_"+ dt_alg.iloc[jj][1] + "-" + dt_alg.iloc[jj][2][:10] + "_" + dt_alg.iloc[jj][0] + "_" + str(jj+1))

    with open(analyze_dir + '03_file-diff/01_filename/filename-listof_'+ algorithms[da] + '.txt', mode='wt', encoding="utf-8") as myfile:
        myfile.write('\n'.join(filename))

print ("Filename_<algorithm-name>.txt has been created")

# Extracting the buggylines of files

Loading txt filename

In [ ]:
fn_hist = (open(analyze_dir + '03_file-diff/01_filename/filename-listof_histogram.txt')).read().split('\n')
fn_min = (open(analyze_dir + '03_file-diff/01_filename/filename-listof_minimal.txt')).read().split('\n')
fn_myers = (open(analyze_dir + '03_file-diff/01_filename/filename-listof_myers.txt')).read().split('\n')
fn_pat = (open(analyze_dir + '03_file-diff/01_filename/filename-listof_patience.txt')).read().split('\n')

In [ ]:
f_names = [fn_hist, fn_min, fn_myers, fn_pat]

Buggylines extraction

In [ ]:
buggylinefiles = []
for n, d_algo in enumerate(data_algorithm):
    bf = []
    for aa in range(0,len(d_algo)):
        sys.stdout.write('\r%i: ' %(n+1) + ' Extracting data %i' %(aa+1))
        sys.stdout.flush()
        name = analyze_dir + "03_file-diff/02_buggylines/" + algorithms[n] + "/" + f_names[n][aa] + "_" + algorithms[n] + "buglines_" + str(aa+1) + ".diff"
        diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=" + algorithms[n] + " " + d_algo.iloc[aa][2] + " " + d_algo.iloc[aa][1] + " -- " + d_algo.iloc[aa][3] + " | grep '^[-]' | grep -Ev '^(--- a/|\+\+\+ b/)' > " + name
        execute_command(diff_cmd, repository)
        zzz = [d_algo.iloc[aa][0], d_algo.iloc[aa][1], d_algo.iloc[aa][2], d_algo.iloc[aa][3], (f_names[n][aa] + "_" + algorithms[n] + "buglines_" + str(aa+1) + ".diff")]
        bf.append(zzz)
    bf = sorted(bf, key=operator.itemgetter(4,1,2,0))
    buggylinefiles.append(bf)
print('\nExtraction is complete')

In [ ]:
buggylinefiles[0]

Reading files of buggylines for each algorithm

In [ ]:
bug_file = []
for m, algrt in enumerate(algorithms):
    temp_file = []
    s = 0
    for f_name in glob.iglob(analyze_dir + '03_file-diff/02_buggylines/' + algrt + '/*', recursive=True):
        sys.stdout.write("\r%i " %(m+1) + "Counting bugline for file %i" %(s+1))
        sys.stdout.flush()
        s += 1
        
        temp_file.append(f_name)
    temp_file.sort()
    bug_file.append(temp_file)
print ("\nReading files are complete")

# Counting number of files and number of buggylines

Removing files with no deletion changes

In [ ]:
devs = []
for n, algr in enumerate(algorithms):
    total = []
    dev = []
    for rr, f_name in enumerate(bug_file[n]):
        sys.stdout.write("\r%i " %(n+1) + "Counting bugline for file %i" %(rr+1))
        sys.stdout.flush()
        
        fn = open(f_name, encoding="utf8", errors='ignore')
        fn = fn.read().split('\n')
        if "--- /dev" in fn[0]:
            num = str(len(fn)-2)
        else:
            num = str(len(fn)-1)
        
        pattern = re.search("((?<=" + algr + "\/)_.\w+[a-zA-Z0-9-.]+(?=_[a-z0-9]))", f_name)
        pattern = pattern.group()[1:]
        
        if num != '0':
            current = [buggylinefiles[n][rr][0], buggylinefiles[n][rr][1], buggylinefiles[n][rr][2], buggylinefiles[n][rr][3], pattern, num]
            total.append(current)
    
    total = sorted(total, key=operator.itemgetter(4,1,2,0))
    
    with open(analyze_dir + "03_file-diff/03_number_of_files/" + algr + "_total_bugline.csv", 'w') as myfile:
        writers = csv.writer(myfile)
        fields = ['bug_id','bugintro_commitID','parent_id','filepath','filename','number_of_buggyline']
        writers.writerow(fields)
        for row in total:
            writers.writerow(row)
            
print ("\nCounting buggylines are complete")

Loading files total_bugline

In [ ]:
df_totalhist = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/histogram_total_bugline.csv')
df_totalmin = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/minimal_total_bugline.csv')
df_totalmyers = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/myers_total_bugline.csv')
df_totalpat = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/patience_total_bugline.csv')

Grouping same files and sum of the number of buglines

In [ ]:
d_frame = [df_totalhist, df_totalmin, df_totalmyers, df_totalpat]
for d, dfr in enumerate(d_frame):
    dframe = (pd.to_numeric(dfr['number_of_buggyline'], errors='coerce')
           .groupby(dfr['filepath'])
           .sum()
           .to_frame()
           .add_prefix('total')
           .reset_index())

    col = ['filepath','totalnumber_of_buggyline']
    dframe = dframe.dropna(subset=['totalnumber_of_buggyline'])
    dframe = dframe.sort_values('totalnumber_of_buggyline', ascending=False)

    dframe[col].to_csv(analyze_dir + '03_file-diff/03_number_of_files/totalbug_of_' + algorithms[d] + '_filechanged.csv')

# Eliminating files with zero total number of buggyline

Loading dataset

In [ ]:
dfrhist = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/totalbug_of_histogram_filechanged.csv')
dfrmin = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/totalbug_of_minimal_filechanged.csv')
dfrmyers = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/totalbug_of_myers_filechanged.csv')
dfrpat = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/totalbug_of_patience_filechanged.csv')

Eliminating zero buggyline files

In [ ]:
cols = ['filepath','totalnumber_of_buggyline']
filehist = dfrhist[cols][dfrhist['totalnumber_of_buggyline'] != 0]
filemin = dfrmin[cols][dfrmin['totalnumber_of_buggyline'] != 0]
filemyers = dfrmyers[cols][dfrmyers['totalnumber_of_buggyline'] != 0]
filepat = dfrpat[cols][dfrpat['totalnumber_of_buggyline'] != 0]

In [ ]:
filehist

In [ ]:
filehist.to_csv(analyze_dir + '03_file-diff/03_number_of_files/01_files_with_buglines/histogramfiles_with_buglines.csv')
filemin.to_csv(analyze_dir + '03_file-diff/03_number_of_files/01_files_with_buglines/minimalfiles_with_buglines.csv')
filemyers.to_csv(analyze_dir + '03_file-diff/03_number_of_files/01_files_with_buglines/myersfiles_with_buglines.csv')
filepat.to_csv(analyze_dir + '03_file-diff/03_number_of_files/01_files_with_buglines/patiencefiles_with_buglines.csv')

Counting number of files

In [ ]:
stat = []
for n, algr in enumerate(algorithms):
    file = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/01_files_with_buglines/' + algr + 'files_with_buglines.csv')
    file = file[cols]
    numfile = len(file)
    totbug = file['totalnumber_of_buggyline'].sum()
    temp = [algr, numfile, totbug]
    stat.append(temp)
print ("Counting files and number of buglines are complete")

In [ ]:
with open(analyze_dir + '03_file-diff/03_number_of_files/01_files_with_buglines/file_and_bug-stats.csv','w') as statfile:
    column = ['algorithm', 'number_of_files', 'totalnumber_of_buglines']
    writers = csv.writer(statfile, delimiter=',')
    writers.writerow(column)
    for item in stat:
        writers.writerow(item)
    
stats = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/01_files_with_buglines/file_and_bug-stats.csv')
stats = stats[column]
stats

Summarizing total number of buglines

# Visualizing number of files and total number of buglines

In [ ]:
numfiles = go.Bar(
    x = stats['algorithm'],
    y = stats['number_of_files'],
    text = stats['number_of_files'],
    textposition = 'auto',
    opacity = 0.6,
    marker = dict(
        color = 'rgb(150,255,200)',
        line = dict(
            color='rgb(8,48,107)',
            width=1.5),
    )
)

dtbar = [numfiles]

layout = go.Layout(
    title = 'Number of files changed in PIG Project'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dtbar, layout=layout)
iplot(fig, show_link=False)

In [ ]:
totbug = go.Bar(
    x = stats['algorithm'],
    y = stats['totalnumber_of_buglines'],
    text = stats['totalnumber_of_buglines'],
    textposition = 'auto',
    name = 'Number of buglines',
    opacity = 0.6,
    marker = dict(
        color = 'rgb(150,255,200)',
        line = dict(
            color='rgb(8,48,107)',
            width=1.5),
    )
)

dtbar = [totbug]

layout = go.Layout(
    title = 'Total number of buglines in PIG Project'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dtbar, layout=layout)
iplot(fig, show_link=False)